# Homework 2

Implement ECDSA from scratch. You want to use the secp256k1 curve. See here for a reference library: https://www.rareskills.io/post/generate-ethereum-address-from-private-key-python

1) pick a private key

2) generate the public key using that private key (not the eth address, the public key)

3) pick message m and hash it to produce h (h can be thought of as a 256 bit number)

4) sign m using your private key and a randomly chosen nonce k. produce (r, s, h, PubKey)

5) verify (r, s, h, PubKey) is valid

You may use a library for point multiplication, but everything else you must do from scratch

In [580]:
from ecpy.curves import Curve, Point
from ecpy.keys import ECPublicKey, ECPrivateKey
from sha3 import keccak_256
import random


In [581]:
# private key is a number

cv = Curve.get_curve('secp256k1')
# curve generator and prime
G_x = 55066263022277343669578718895168534326250603453777594175500187360389116729240
G_y = 32670510020758816978083085130507043184471273380659243275938904335757337482424
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663

private_key = random.randint(0, cv.field)

pv_key = ECPrivateKey(private_key, cv)
pu_key = pv_key.get_public_key()

pub_key = cv.mul_point(private_key , cv.generator)
#compare generated vs calculated public key
print(pub_key.x == pu_key.W.x and pub_key.y == pu_key.W.y)

#concatenate x and y of the public key then take the last 20 bytes of the hash
concat_x_y = pu_key.W.x.to_bytes(32, byteorder='big') + pu_key.W.y.to_bytes(32, byteorder='big')
eth_addr = '0x' + keccak_256(concat_x_y).digest()[-20:].hex()

print(private_key)
print('private key: ', hex(private_key))

print('eth_address: ', eth_addr)

True
68593936611348105543962362737492430222342372080500869738209197383858565885651
private key:  0x97a6cad3c968c5342e4bc3bbde63115ef54dd19c160d7e65ba75145eed1b2ed3
eth_address:  0x89cadbb52d30c8d4c9acc4c3d83397ca9b300ddf


# Hashing the message

In [582]:
m = "message to hash"
h = keccak_256(m.encode())

prefix = b"\x19Ethereum Signed Message:\n32"
message_to_sign_bytes = (prefix + h.digest())

print(message_to_sign_bytes)
eth_message = keccak_256(message_to_sign_bytes)

print('0x' + eth_message.hexdigest())

b'\x19Ethereum Signed Message:\n32\x1cp\x9cR\xcd\xb0\xe2\x81\xd3\xf5\xa0\xe1\xad\xdd\x90\xe9\x87\x83"~\xa8T3\n\x8c)\xa1\xbb3Kq\xab'
0x1fff06355a39f9d0aa37185a7ce8686ea94eca6297a0bd6c670699578471d496


# Signing the message
1) Calculate a hash (z) from the message to sign.
2) Generate a secure random value for k (our nonce).
3) Calculate point (x₁, y₁) on the elliptic curve by multiplying k with the G constant of the elliptic curve.
4) Calculate r = x₁ mod n. If r equals zero, go back to step 2.
5) Calculate s = k⁻¹(z + rdₐ) mod n. If s equals zero, go back to step 2.
6) Determine v = 27 is y1 is even, 28 if odd

In [590]:
r = 0
s = 0
v = 28

z = int(eth_message.hexdigest(),16)
n = cv.order
g = cv.generator

while ( r == 0 or s == 0):
    k = random.randint(0, n)
    x1 = (k * g).x
    y1 = (k * g).y
    if(y1 % 2 == 0):
        v = 27
    
    r = x1 % n
    inv_k = pow(k,-1,n)
    s = inv_k * (z + r * private_key) % n

print("r ", "0x" + r.to_bytes(32).hex())
print("s ", "0x" + s.to_bytes(32).hex())
print("v ", v)

r  0x7ee9e2f57834467f365f7e22221f5663c24175e89bc58d617d786bd80320274f
s  0x315685a6fee86c1402882c6036f0ed74be867a6c8def54910a99c5fe9427c98d
v  28


# The process for recovering the public key looks like this:

1) Calculate the hash (e) for the message to recover.
2) Calculate point R = (x₁, y₁) on the elliptic curve, where x₁ is r. y1 is recovered from x1 (= x1 if v is 27, then it would be symmetry over x if v is 28)
3) Calculate u₁ = -zr⁻¹ mod n and u₂ = sr⁻¹ mod n.
4) Calculate point Qₐ = (xₐ, yₐ) = u₁ × G + u₂ × R.
Qₐ is the point of the public key for the private key that the address was signed with. We can derive an address from this and check if that matches with the provided address. If it does the signature is valid.

In [591]:

R_x = r

R_y = cv.y_recover(R_x, 0)
if(v==28):
    R_y = cv.y_recover(R_x, 1)

R = Point(R_x, R_y, cv)

r_inv = pow(r, -1, n)

u_1 = (-1 * z * r_inv) % n
u_2 = (s * r_inv) % n


Qa = cv.add_point(cv.mul_point(u_1 , cv.generator), cv.mul_point(u_2 , R))


#concatenate x and y of the public key then take the last 20 bytes of the hash
concat_x_y = Qa.x.to_bytes(32, byteorder='big') + Qa.y.to_bytes(32, byteorder='big')
recovered_eth_addr = '0x' + keccak_256(concat_x_y).digest()[-20:].hex()
print(recovered_eth_addr == eth_addr)

True


In [596]:
R_x = r

R_y = cv.y_recover(R_x, 0)
if(v==28):
    R_y = cv.y_recover(R_x, 1)


R = Point(R_x, R_y, cv)

s_inv = pow(s, -1, n)

u_1 = (z * s_inv) % n
u_2 = (r * s_inv) % n


Qa = cv.add_point(cv.mul_point(u_1 , cv.generator), cv.mul_point(u_2 , R))

#concatenate x and y of the public key then take the last 20 bytes of the hash
concat_x_y = Qa.x.to_bytes(32, byteorder='big') + Qa.y.to_bytes(32, byteorder='big')
recovered_eth_addr = '0x' + keccak_256(concat_x_y).digest()[-20:].hex()
print(recovered_eth_addr == eth_addr)

False
